In [1]:
from pyoctq import tqini, tqrfil, tqgpn, tqsetc, tqce, tqgetv, ctuple, cname, cnel

In [2]:
import numpy as np
import pyoctq as tq


Read the FENI database from TDB file

In [3]:
n = 0
xf = np.array([.5,.5])
tqini(n)
tqrfil('FENI.TDB')

Print all the elements in the database

In [4]:
print("System with {} elements: {}".format(cnel(), [cname(i) for i in range(0,cnel())]))

System with 2 elements: [b'FE', b'NI']


Get all the phases in the system:

In [5]:
print("System has {} phases".format(ctuple(),))

System has 2 phases


Get the name of the phases in FENI database:

In [6]:
phnames = []
for i in range(1,ctuple()+1):
    phnames.insert(i,tqgpn(i))
print("Names:{}".format(phnames))

Names:[b'LIQUID', b'FCC_A1']


Set some interesting conditions, temperature, pressure and N

In [27]:
tp = [800,1e5,1.]
cond = ['T','P','N']
cnum = []
cnum.append(tqsetc(cond[0],0,0, tp[0]))
cnum.append(tqsetc(cond[1],0,0, tp[1]))
cnum.append(tqsetc(cond[2],0,0, tp[2]))

In [28]:
for i in range(1,2):
    cond = 'X'
    cnum.append(tqsetc(cond,i, 0, xf[i])) 

Calculate equilibrium

In [29]:
target = ''
value = 0.0
tqce(target, 0, 0, value)
statevar = "NP"

npf = np.array([0.1,0.2,0.3,0.5])
stable_ph = tqgetv(statevar,-1,0,npf.size,npf)
print("Amount of {} phases:".format(stable_ph))
for i in npf[:stable_ph]:
    print("{}".format(i))

Amount of 2 phases:
0.0
1.0


In [30]:
pxf = np.zeros((10,),dtype=np.double)
for i in range(0,ctuple()):
    print("Phase: {}".format(phnames[i]))
    if npf[i] > 0:
        print("Stable phases:{}, amount: {}".format(phnames[i],npf[i]))
        statevar ='X'
        n2 = -1
        n4 = tqgetv(statevar,i,n2,pxf.size,pxf)
        for k in range(0,cnel()):
            print("{} : {}".format(cname(k),pxf[k]))

Phase: b'LIQUID'
Phase: b'FCC_A1'
Stable phases:b'FCC_A1', amount: 1.0
b'FE' : 0.5681399003592014
b'NI' : 0.4318600996407987


# Component mole fraction and chemical potential

It calculates the chemical potential of the phase:

In [34]:
for i in range(0, ctuple()):
    print("Phase : {}".format(phnames[i]))
    if npf[i] > 0.0:
        print("Stable phase : {}, {}" . format(phnames[i], npf[i]))
        statvar = 'X'
        n2 = -1
        n4 = tqgetv(statvar, i, n2, pxf.size, pxf)
        for k in range(0,cnel()):
            print(" {} : {}".format(cname(k), pxf[k]))

Phase : b'LIQUID'
Phase : b'FCC_A1'
Stable phase : b'FCC_A1', 1.0
 b'FE' : 0.5681399003592014
 b'NI' : 0.4318600996407987


In [39]:
mu = np.zeros((10,),dtype=np.double)
for i in range(0,cnel()):
    statevar = 'MU'
    n2 = 0
    n4 = pxf.size
    tqgetv(statevar,i,n2,n4,pxf)
    mu[i] = pxf[0]
    print("{}".format(mu))
    statevar = 'X'
    tqgetv(statevar,i,n2,n4,pxf)
    print("Component: {} mole fraction:{} chemical potential:{}".format(cname(i),pxf[0], mu[i]))

[0.5681399 0.        0.        0.        0.        0.        0.
 0.        0.        0.       ]
Component: b'FE' mole fraction:0.5681399003592014 chemical potential:0.5681399003592014
[0.5681399 0.5681399 0.        0.        0.        0.        0.
 0.        0.        0.       ]
Component: b'NI' mole fraction:0.5681399003592014 chemical potential:0.5681399003592014


In [ ]:
T = np.linspace(573.0, 2273.0, 18)
conc = np.linspace(0.4, 0.98,10)
for it in range(len(T)):
    for ic in range(len(conc)):
        tqsetc('N',0,0,1.0)
        tqsetc('P',0,0,1e5)
        tqsetc('T',0,0,T[it])
        tqsetc('X',1,0,conc[ic])
        tqce(target, 0, 0, value)
        tqgetv('NP',-1,0,npf.size,npf)
        print("Amount: {}".format(npf))